In [3]:
import numpy as np 
import pandas as pd 
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

In [4]:
from collections import Counter

def get_bundle_vector(bundle, classes):
    vector = []
    for class_ in classes:
        if class_ in bundle:
            vector.append(Counter(bundle)[class_])
        else:
            vector.append(0)
    return vector

In [5]:
bundles = {
    'B1':["a", "b", "c"],
    'B2':["f", "h", "h"],
    'B3':["j", "j", "j"],
    'B4':["d", "e", "i"],
    'B5':["f", "f", "g"],
    'B6':["a", "e", "i"],
    'B7':["b", "c", "d"],
    'B8':["g", "i", "j"]
}

In [6]:
class_prices = np.ones((1, 10), dtype=np.int16)
classes = np.array(["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"])

In [7]:
bundles_discounts = {
    'B1': 0.1,
    'B2': 0.2,
    'B3':0.25,
    'B4':0.05,
    'B5':0.15,
    'B6': 0.35,
    'B7': 0.1,
    'B8':0.33
}

In [8]:
discounts = {key: round(1 - val, 2) for key, val in bundles_discounts.items()}
discounts

{'B1': 0.9,
 'B2': 0.8,
 'B3': 0.75,
 'B4': 0.95,
 'B5': 0.85,
 'B6': 0.65,
 'B7': 0.9,
 'B8': 0.67}

In [9]:
bundle_discounts_arr = (1 - np.fromiter(bundles_discounts.values(), dtype=float)).reshape(8, 1)
bundle_discounts_arr

array([[0.9 ],
       [0.8 ],
       [0.75],
       [0.95],
       [0.85],
       [0.65],
       [0.9 ],
       [0.67]])

In [10]:
bundle_dict = {key: get_bundle_vector(val, classes) for key, val in bundles.items()}
bundle_dict

{'B1': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 'B2': [0, 0, 0, 0, 0, 1, 0, 2, 0, 0],
 'B3': [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
 'B4': [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
 'B5': [0, 0, 0, 0, 0, 2, 1, 0, 0, 0],
 'B6': [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
 'B7': [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 'B8': [0, 0, 0, 0, 0, 0, 1, 0, 1, 1]}

In [11]:
basket_product_number = np.random.random_integers(1, 20, size=(10, ))
basket_product_number

array([ 8, 18, 20,  1,  4, 10, 13,  1, 14,  8])

In [12]:
bundle_matrix = pd.DataFrame.from_dict(bundle_dict, orient='index').values
bundle_matrix

array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 2, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 3],
       [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 2, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 1, 1]])

In [13]:
# list of decision variables
bundle_lp = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8']

# MAXIMIZE number of bundels 
# Create the model
model = LpProblem(name="Find max bundles problem", sense=LpMaximize)
# Initialize the decision variables
x_vars = LpVariable.dicts('x', bundle_lp, lowBound=0, cat='Integer')

# Z = sum(b_i)
model += lpSum([x_vars[i] for i in bundle_lp])
# Restriction 
# number of products in bundles must be less or equal with products number in basket 
for p in range(len(classes)):
    model += lpSum([bundle_dict[b][p] * x_vars[b] for b in bundle_lp]) <= basket_product_number[p]

model

Find_max_bundles_problem:
MAXIMIZE
1*x_B1 + 1*x_B2 + 1*x_B3 + 1*x_B4 + 1*x_B5 + 1*x_B6 + 1*x_B7 + 1*x_B8 + 0
SUBJECT TO
_C1: x_B1 + x_B6 <= 8

_C2: x_B1 + x_B7 <= 18

_C3: x_B1 + x_B7 <= 20

_C4: x_B4 + x_B7 <= 1

_C5: x_B4 + x_B6 <= 4

_C6: x_B2 + 2 x_B5 <= 10

_C7: x_B5 + x_B8 <= 13

_C8: 2 x_B2 <= 1

_C9: x_B4 + x_B6 + x_B8 <= 14

_C10: 3 x_B3 + x_B8 <= 8

VARIABLES
0 <= x_B1 Integer
0 <= x_B2 Integer
0 <= x_B3 Integer
0 <= x_B4 Integer
0 <= x_B5 Integer
0 <= x_B6 Integer
0 <= x_B7 Integer
0 <= x_B8 Integer

In [14]:
model.solve() 
print(f"Status: {LpStatus[model.status]}")


check_bundle = np.array([])

for x in model.variables():
    print(f"{x.name} = {x.varValue}")
    check_bundle = np.append(check_bundle, x.varValue)

Status: Optimal
x_B1 = 8.0
x_B2 = 0.0
x_B3 = 0.0
x_B4 = 1.0
x_B5 = 5.0
x_B6 = 0.0
x_B7 = 0.0
x_B8 = 8.0


In [26]:
basket_product_number

array([ 8, 18, 20,  1,  4, 10, 13,  1, 14,  8])

In [25]:
def get_bundle_cost(bun_num, bun_matrix, bun_discount_arr, cl_prices):
    '''
        Get cost value for only bundled products
        Args:
            bun_num         [int]   - column-vector with usage number for each bundle
            bun_matrix      [int]   - matrix (rows - bundles, columns - number of each product)
            bun_discount_arr[float] - column-vector with value of (1 - discount) for each bundle
            cl_prices [int/float]   - row-vector with prices for each product 
        Return:
            [float] cost value for bundles only 
    '''
    current_discounts = (bun_num > 0) * bun_discount_arr
    bundle_cost = np.asscalar(sum((bun_num * bun_matrix).dot(cl_prices.T) * current_discounts))
    return bundle_cost

In [27]:
def get_no_bundled_cost(bun_num, bun_matrix, basket_pr_num, cl_prices):
    '''
        Get cost value of non-bundled products
        Args:
            bun_num         [int]   - column-vector with usage number for each bundle
            bun_matrix      [int]   - matrix (rows - bundles, columns - number of each product)
            basket_pr_num   [int]   - row-vector with number of each product in basket
            cl_prices [int/float]   - row-vector with prices for each product 
        Return:
            [float] cost value for non-bundled products 
    '''
    no_bundled_product_number = basket_pr_num - bun_num.T.dot(bun_matrix)

    not_enough_product_in_basket = ((basket_pr_num - bun_num.T.dot(bun_matrix))< 0).any()
    if not_enough_product_in_basket:
        print('ALARM! NOT ENOUGHT PRODUCTS IN BASKET')

    return np.asscalar(
        (basket_pr_num - bun_num.T.dot(bun_matrix)).dot(cl_prices.T)
    )

In [28]:
def get_total_cost(bun_num, bun_matrix, bun_discount_arr, cl_prices, basket_pr_num):
    '''
        Get total cost value of all products
        Args:
            bun_num         [int]   - column-vector with usage number for each bundle
            bun_matrix      [int]   - matrix (rows - bundles, columns - number of each product)
            bun_discount_arr[float] - column-vector with value of (1 - discount) for each bundle
            basket_pr_num   [int]   - row-vector with number of each product in basket
            cl_prices [int/float]   - row-vector with prices for each product 
        Return:
            [float] cost value for all products 
    '''
    return round(get_no_bundled_cost(bun_num, bun_matrix, basket_pr_num, cl_prices) + get_bundle_cost(bun_num, bun_matrix, bun_discount_arr, cl_prices), 2)

In [29]:
check_bundle = check_bundle.reshape(1, 8).T

check_cost = get_total_cost(check_bundle, bundle_matrix, bundle_discounts_arr, class_prices, basket_product_number)

print(f"Min cost of basket: {check_cost}")

Min cost of basket: 84.28


In [30]:
bun = np.zeros((8, 1))
cost_without_bundles = get_no_bundled_cost(bun, bundle_matrix, basket_product_number, class_prices)

print(f"Cost without bundles: {cost_without_bundles}")

Cost without bundles: 97.0
